# $ Classification Exercises $

### imports

In [10]:
import pandas as pd
import env
from pydataset import data

# 4.  Create a pandas dataframe, df_iris, from this data.

In [14]:
# use pydataset to import iris data and create data frame
df = data('iris')

# show top 3 rows of iris data
df.head(3)


,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
1,5.1,3.5,1.4,0.2,setosa
2,4.9,3.0,1.4,0.2,setosa
3,4.7,3.2,1.3,0.2,setosa


In [15]:
# use attribute shape of dataframe to obtain number of rows and columns
df.shape

(150, 5)

In [16]:
# use attribute columns to obtain column names
df.columns

Index(['Sepal.Length', 'Sepal.Width', 'Petal.Length', 'Petal.Width',
       'Species'],
      dtype='object')

In [19]:
# use attribute dtypes to see data types of columns
df.dtypes

Sepal.Length    float64
Sepal.Width     float64
Petal.Length    float64
Petal.Width     float64
Species          object
dtype: object

In [21]:
# use describe method to see useful statistics of data
df.describe()

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.057333,3.758000,1.199333
std,0.828066,0.435866,1.765298,0.762238
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


### Would you recommend rescaling the data based on these statistics?
At this moment there is no recommendation of rescaling data since all the measurements seem to be in the same unit of measurement.

# 5. Read the data from this google sheet into a dataframe, df_google.

In [44]:
# save url to variable
sheet_url = 'https://docs.google.com/spreadsheets/d/1Uhtml8KY19LILuZsrDtlsHHDC9wuDGUSe8LTEwvdI5g/edit#gid=341089357'
# replace edit with export and save to variable
csv_export_url = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')

In [47]:
# read url created csv and create data frame
google_df = pd.read_csv(csv_export_url)

# show top 3 rows fo data frame
google_df.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [48]:
# use attribute shape of dataframe to obtain number of rows and columns
google_df.shape

(891, 12)

In [49]:
# use attribute columns to obtain column names
google_df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [50]:
# use attribute dtypes to see data types of columns
google_df.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [96]:
# use describe method to see useful statistics of numeric variables
google_df.select_dtypes('int64','float64').describe()

,PassengerId,Survived,Pclass,SibSp,Parch
count,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,0.523008,0.381594
std,257.353842,0.486592,0.836071,1.102743,0.806057
min,1.000000,0.000000,1.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,0.000000,0.000000
50%,446.000000,0.000000,3.000000,0.000000,0.000000
75%,668.500000,1.000000,3.000000,1.000000,0.000000
max,891.000000,1.000000,3.000000,8.000000,6.000000


In [97]:
# use nunique method to print how many unique values are in categorical variables
google_df.select_dtypes('object').nunique()

Name        891
Sex           2
Ticket      681
Cabin       147
Embarked      3
dtype: int64

# 6. Download the previous exercise's file into an excel (File → Download → Microsoft Excel). Read the downloaded file into a dataframe named df_excel.

In [77]:
# create data frame from excel file
df_excel = pd.read_excel('df_excel.xlsx')
df_excel_sample = pd.read_excel('df_excel.xlsx').head(100)

In [78]:
# return the number of rows only using attribute shape
df_excel.shape[0]

891

In [82]:
# print the first 5 column names using attribute [:5] (inclusive) of data frame
df_excel_sample.columns[:5]

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex'], dtype='object')

In [95]:
# print of column names that have a data type of object
df_excel_sample.select_dtypes(object).columns

Index(['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked'], dtype='object')

In [108]:
# compute the range of numeric variables in data frame
df_excel_sample.select_dtypes('float64').max()-df_excel_sample.select_dtypes('float64').min()

PassengerId     99.000
Survived         1.000
Pclass           2.000
Age             70.170
SibSp            5.000
Parch            5.000
Fare           255.775
dtype: float64